In [1]:
with open("file.bin", "wt") as f: # открываем файл с помощью with
    f.write("abcdefg")

Чтобы написать контекстный менеджер, нужно всего лишь помнить о нескольких вещах:

Нужно создать класс и написать в нём метод __enter__. Код в этом методе будет выполняться при входе в контекстный менеджер (при создании объекта с ключевым словом with).

Написать метод __exit__. Этот метод будет выполнять код, помещённый в него, на выходе.

Добавить в этот метод три дополнительных аргумента помимо self — exc_type, exc_val, exc_tb. Зачем они нужны, расскажу чуть позже.

In [2]:
from datetime import datetime
import time # проверять действие измерителя будем с помощью библиотеки time
 
 
# вся суть этого измерителя заключается в том, что мы считаем разницу в секундах между открытием и закрытием контекстного менеджера
class Timer:
    def __init__(self): 
            pass
 
    def __enter__(self): # этот метод вызывается при запуске с помощью with. Если вы хотите вернуть какой-то объект, чтобы потом работать с ним в контекстном менеджере, как в примере с файлом, то просто верните этот объект через return
            self.start = datetime.utcnow()
            return None
    
    def __exit__(self, exc_type, exc_val, exc_tb): # этот метод срабатывает при выходе из контекстного менеджера
            print(f"Time passed: {(datetime.utcnow() - self.start).total_seconds()}")
 
 
with Timer():
    time.sleep(2)  # засыпаем на 2 секунды

Time passed: 2.002215


exc_type — это тип исключения, из-за которого вылетел контекстный менеджер. Если всё прошло успешно, то значение этого аргумента будет None.

exc_val — сообщение в исключении. Аналогично: если всё прошло успешно, этот аргумент будет None.

exc_tb — объект сообщения от интерпретатора. Лучшего его вообще не трогать, если вы не разработчик языка, но тем не менее он всегда ждёт вас здесь. Возможно когда-то, после нашего курса вы…..

Как можно было догадаться, Python был бы не Python, если бы у него не было библиотек на любой крайний случай. В далёкой версии Python 2.6 появилась возможность создания контекстных менеджеров через генераторы.

Давайте взглянем, как это можно сделать.

Но перед этим хотелось бы разъяснить разницу в реализации. По сути, здесь разница только в том, что контекстный менеджер на генераторах — это функция (для особо привередливых — генератор). В ней до yield выполняется код, который мы могли бы записать в __enter__, если бы делали контекстный менеджер в виде класса, а после yield пишем код, который выполнился бы в __exit__. То есть до yield — всё что произойдёт при входе, после — всё, что на выходе. Вот и вся разница. Как видите, она не велика.

In [3]:
from datetime import datetime
import time
 
 
from contextlib import contextmanager # импортируем нужный нам декоратор
 
 
@contextmanager # оборачиваем функцию в декоратор contextmanager
def timer():
    start = datetime.utcnow()
    yield # если вам нужно что-то вернуть через контекстный менеджер, просто вставьте этот объект сюда.
    print(f"Time passed: {(datetime.utcnow() - start).total_seconds()}")
 
 
with timer():
    time.sleep(2)

Time passed: 2.002228


Задание 8.5.6

Напишите контекстный менеджер, который умеет безопасно работать с файлами.

При входе в контекстный менеджер передаются два аргумента: первый — путь к файлу который надо открыть, второй — тип открываемого файла (для записи, для чтения и т. д.). При выходе из контекстного менеджера файл должен закрываться. (Эталоном работы можно считать контекстный менеджер open).

In [4]:
class OpenFile:
    def __init__(self, path, type):
        self.file = open(path, type)
    
    def __enter__(self):
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file.close()
 
 
with OpenFile('hello.txt', 'wt') as f:
    f.write('Мой контекстный менеджер делает тоже самое!')